In [1]:
import pandas as pd
import numpy as np
import warnings
# Disable warnings
warnings.filterwarnings("ignore")

## Extracting features of 2022 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df = pd.read_html(link, header=0)[1]
df

Rank                                        Title            Distributor  \
0     1                            Top Gun: Maverick              Paramount   
1     2                     Avatar: The Way of Water  20th Century / Disney   
2     3               Black Panther: Wakanda Forever                 Disney   
3     4  Doctor Strange in the Multiverse of Madness                 Disney   
4     5                      Jurassic World Dominion              Universal   
5     6                     Minions: The Rise of Gru              Universal   
6     7                                   The Batman           Warner Bros.   
7     8                       Thor: Love and Thunder                 Disney   
8     9                         Sonic the Hedgehog 2              Paramount   
9    10                 Puss in Boots: The Last Wish              Universal   

  Domestic gross  
0   $718,732,821  
1   $684,075,767  
2   $453,829,060  
3   $411,331,607  
4   $376,851,080  
5   $369,695,210  
6   $369,345,583  
7   $343,256,830  
8   $190,872,904  
9   $185,535,345

In [4]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [5]:
df

Opening  Opening.1                     Title  \
0      J A N U A R Y        7.0                   The 355   
1      J A N U A R Y        7.0  The Legend of La Llorona   
2      J A N U A R Y        7.0              The Commando   
3      J A N U A R Y        7.0            American Siege   
4      J A N U A R Y       14.0                    Scream   
..               ...        ...                       ...   
313  D E C E M B E R       30.0            Alice, Darling   
314  D E C E M B E R        NaN                       NaN   
315  D E C E M B E R        NaN                       NaN   
316  D E C E M B E R        NaN                       NaN   
317  D E C E M B E R        NaN                       NaN   

                                    Production company  \
0    Universal Pictures / Freckle Films / FilmNatio...   
1                       Saban Films / Ageless Pictures   
2                 Saban Films / Premiere Entertainment   
3                               Vertical Entertainment   
4    Paramount Pictures / Spyglass Media Group / Ra...   
..                                                 ...   
313                     Lionsgate / Elevation Pictures   
314                                                NaN   
315                                                NaN   
316                                                NaN   
317                                                NaN   

                                         Cast and crew   Ref.  
0    Simon Kinberg (director/screenplay); Theresa R...    [2]  
1    Patricia Harris Seeley (director); José Prende...    [3]  
2    Asif Akbar (director); Koji Steven Sakai (scre...    [4]  
3    Edward John Drake (director/screenplay); Timot...    [5]  
4    Matt Bettinelli-Olpin, Tyler Gillett (director...    [6]  
..                                                 ...    ...  
313  Mary Nighy (director); Alanna Francis (screenp...  [261]  
314                                                NaN    NaN  
315                                                NaN    NaN  
316                                                NaN    NaN  
317                                                NaN    NaN  

[318 rows x 6 columns]

In [6]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'bd9d93e4649f8d7b42731beda117f8c6'
from tmdbv3api import Movie
tmdb_movie = Movie()
from tqdm import tqdm
tqdm.pandas()

In [7]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " "
            for i in range(len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

# Modify the function to print the index and title, and handle the error
def safe_get_genre(title, index):
    try:
        return get_genre(title)
    except Exception as e:
        print(f"Error processing index: {index}")
        print(f"Title: {title}")
        print(f"Error: {e}")
        return np.NaN
        
# Apply the function with progress bar
df['genres'] = df.progress_apply(lambda row: safe_get_genre(str(row['Title']), row.name), axis=1)

 22%|█████████████████▊                                                               | 70/318 [01:44<05:27,  1.32s/it]

Error processing index: 68
Title: Apollo 10 1⁄2: A Space Age Childhood
Error: attribute name must be string, not 'int'


100%|████████████████████████████████████████████████████████████████████████████████| 318/318 [08:09<00:00,  1.54s/it]


In [8]:
def get_id(x):
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        return movie_id

# Modify the function to print the index and title, and handle the error
def safe_get_id(title, index):
    try:
        return get_id(title)
    except Exception as e:
        print(f"Error processing index: {index}")
        print(f"Title: {title}")
        print(f"Error: {e}")
        return np.NaN

# Apply the function with progress bar
df['id'] = df.progress_apply(lambda row: safe_get_id(str(row['Title']), row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 318/318 [00:00<00:00, 3798.02it/s]

Error processing index: 68
Title: Apollo 10 1⁄2: A Space Age Childhood
Error: attribute name must be string, not 'int'


In [9]:
df

Opening  Opening.1                     Title  \
0      J A N U A R Y        7.0                   The 355   
1      J A N U A R Y        7.0  The Legend of La Llorona   
2      J A N U A R Y        7.0              The Commando   
3      J A N U A R Y        7.0            American Siege   
4      J A N U A R Y       14.0                    Scream   
..               ...        ...                       ...   
313  D E C E M B E R       30.0            Alice, Darling   
314  D E C E M B E R        NaN                       NaN   
315  D E C E M B E R        NaN                       NaN   
316  D E C E M B E R        NaN                       NaN   
317  D E C E M B E R        NaN                       NaN   

                                    Production company  \
0    Universal Pictures / Freckle Films / FilmNatio...   
1                       Saban Films / Ageless Pictures   
2                 Saban Films / Premiere Entertainment   
3                               Vertical Entertainment   
4    Paramount Pictures / Spyglass Media Group / Ra...   
..                                                 ...   
313                     Lionsgate / Elevation Pictures   
314                                                NaN   
315                                                NaN   
316                                                NaN   
317                                                NaN   

                                         Cast and crew   Ref.  \
0    Simon Kinberg (director/screenplay); Theresa R...    [2]   
1    Patricia Harris Seeley (director); José Prende...    [3]   
2    Asif Akbar (director); Koji Steven Sakai (scre...    [4]   
3    Edward John Drake (director/screenplay); Timot...    [5]   
4    Matt Bettinelli-Olpin, Tyler Gillett (director...    [6]   
..                                                 ...    ...   
313  Mary Nighy (director); Alanna Francis (screenp...  [261]   
314                                                NaN    NaN   
315                                                NaN    NaN   
316                                                NaN    NaN   
317                                                NaN    NaN   

                              genres        id  
0                    Action Thriller  522016.0  
1    Family Animation Fantasy Horror  116322.0  
2              Action Crime Thriller  753232.0  
3          Action Adventure Thriller  766907.0  
4            Horror Mystery Thriller  646385.0  
..                               ...       ...  
313           Thriller Drama Romance  850028.0  
314                      Drama Crime  384717.0  
315                      Drama Crime  384717.0  
316                      Drama Crime  384717.0  
317                      Drama Crime  384717.0  

[318 rows x 8 columns]

In [10]:
df_2022 = df[['Title','Cast and crew','genres','id']]

In [11]:
df_2022

Title  \
0                     The 355   
1    The Legend of La Llorona   
2                The Commando   
3              American Siege   
4                      Scream   
..                        ...   
313            Alice, Darling   
314                       NaN   
315                       NaN   
316                       NaN   
317                       NaN   

                                         Cast and crew  \
0    Simon Kinberg (director/screenplay); Theresa R...   
1    Patricia Harris Seeley (director); José Prende...   
2    Asif Akbar (director); Koji Steven Sakai (scre...   
3    Edward John Drake (director/screenplay); Timot...   
4    Matt Bettinelli-Olpin, Tyler Gillett (director...   
..                                                 ...   
313  Mary Nighy (director); Alanna Francis (screenp...   
314                                                NaN   
315                                                NaN   
316                                                NaN   
317                                                NaN   

                              genres        id  
0                    Action Thriller  522016.0  
1    Family Animation Fantasy Horror  116322.0  
2              Action Crime Thriller  753232.0  
3          Action Adventure Thriller  766907.0  
4            Horror Mystery Thriller  646385.0  
..                               ...       ...  
313           Thriller Drama Romance  850028.0  
314                      Drama Crime  384717.0  
315                      Drama Crime  384717.0  
316                      Drama Crime  384717.0  
317                      Drama Crime  384717.0  

[318 rows x 4 columns]

In [12]:
def get_director(x):
    if " (director)" in x:
        return x.split("(director)")[0].split(";")[-1]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (director/screenplay)" in x:
        return x.split("(director/screenplay)")[0]
    elif "(directors/screenplay); " in x:
        return x.split("(directors/screenplay); ")[0]
    else:
        return x.split("(director/​screenplay)")[0]

In [13]:
df_2022['director_name'] = df_2022['Cast and crew'].astype("str").map(lambda x: get_director(x))

In [14]:
t=0
for i in df_2022['director_name']:
    t=t+1
    print(t,i)

1 Simon Kinberg 
2 Patricia Harris Seeley 
3 Asif Akbar 
4 Edward John Drake 
5 Matt Bettinelli-Olpin, Tyler Gillett 
6 Derek Drymon, Jennifer Kluska
7 Luis Prieto 
8 Rick Jacobson 
9 D. J. Caruso 
10 Sean McNamara 
11 Ray Giarratana 
12 Justin Lee 
13 Megan Park 
14 Charles Kinnane, Daniel Kinnane 
15 John C. Donkin 
16 Paul Solet 
17 Jeff Tremaine 
18 Roland Emmerich 
19 Tim Kirkby 
20 Rich Ragsdale 
21 Steven Soderbergh 
22 D. J. Caruso 
23 Kenneth Branagh 
24 Kat Coiro 
25 Mark Williams 
26 Jason Orley 
27 Emily Ting 
28 Josephine Decker 
29 Arie Posin 
30 Roel Reiné 
31 Ruben Fleischer 
32 Channing Tatum 
33 David Blue Garcia 
34 Sean Ellis 
35 Erik Peter Carlson 
36 Joe Wright 
37 B. J. McDonnell 
38 Tyler Perry 
39 Damien Power 
40 Martha Coolidge 
41 Edward John Drake 
42 Matt Reeves 
43 Kogonada 
44 Mimi Cave 
45 Wes Miller 
46 Jon McDonald 
47 Shawn Levy 
48 Domee Shi 
49 Kim Bass 
50 Katt Shea 
51 Graham Moore 
52 Adrian Lyne 
53 Gail Lerner 
54 Charlie McDowell 
55 Ti West 

In [15]:
def get_actor1(x):
    if "(screenplay); " in x:
        return ((x.split("(screenplay); ")[-1]).split(", ")[0]) 
    elif " (director/screenplay);"in x:
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0].split(";")[-1])
    elif " (directors/screenplay);"in x:
        return ((x.split("(directors/screenplay);")[-1]).split(", ")[0]) 
    else:
        return ((x.split('(director); ')[-1]).split(",")[0])
     

In [16]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].astype("str").map(lambda x: get_actor1(x))

In [17]:
t=0
for i in df_2022['actor_1_name']:
    t=t+1
    print(t,i)

1 Jessica Chastain
2 Autumn Reeser
3 Mickey Rourke
4 Timothy V. Murphy
5 Melissa Barrera
6 Andy Samberg
7 Cameron Monaghan
8 Laura Marano
9 Abigail Cowen
10 Pierce Brosnan
11 Christian Convery
12 Bruce Dern
13 Jenna Ortega
14 Kevin James
15 Simon Pegg
16 Adrien Brody
17 Johnny Knoxville
18 Halle Berry
19 Charlie Hunnam
20 Scout Taylor-Compton
21 Zoë Kravitz
22 Rainey Qualley
23 Tom Bateman
24 Jennifer Lopez
25 Liam Neeson
26 Charlie Day
27 Ava Michelle
28 Grace Kaufman
29 Joey King
30 Iko Uwais
31 Tom Holland
32 Channing Tatum
33 Sarah Yarkin
34 Boyd Holbrook
35 Emily Shenaut
36 Peter Dinklage
37 Dave Grohl
38 Tyler Perry
39 Havana Rose Liu
40 Adelaide Clemens
41 Devon Sawa
42 Robert Pattinson
43 Colin Farrell
44 Daisy Edgar-Jones
45 Kevin Dillon
46 Cary Elwes
47 Ryan Reynolds
48 Rosalie Chiang
49 Rory Cochrane
50 Grant Gustin
51 Mark Rylance
52 Ben Affleck
53 Gabrielle Union
54 Lily Collins
55 Mia Goth
56 Sandra Oh
57 Keke Palmer
58 Regina Hall
59 Melissa Leo
60 Sandra Bullock
61  Mic

In [18]:
def get_actor2(x):
    if "(screenplay); " in x:
        if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
            return np.NaN
        else:
            return ((x.split("screenplay); ")[-1]).split(", ")[1])
            
    elif "(directors);"in x:
        if len((x.split("(directors);")[-1]).split(", ")) >2:
            return ((x.split("(directors);")[-1]).split(", ")[1])
        else:
            np.NaN
            
    elif "(director/screenplay);"in x:
        if len((x.split("(director/screenplay);")[-1]).split(", ")) >2:
            return ((x.split("(director/screenplay);")[-1]).split(", ")[1])
        else:
            np.NaN
    elif "(director); "in x:
        if len((x.split("(director); ")[-1]).split(", ")) >2:
            return ((x.split("(director); ")[-1]).split(", ")[1])
        else:
            np.NaN
           
    elif " (directors/screenplay);"in x:
        if len((x.split("(directors/screenplay);")[-1]).split(", ")) >2:
            return ((x.split("(directors/screenplay);")[-1]).split(", ")[1])
        else:
            np.NaN
    else:
        if len(x.split('(director); ')) > 2:
            return ((x.split('(director); ')[-1]).split(",")[0])
        else:
            np.NaN


In [19]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].astype("str").map(lambda x: get_actor2(x))

In [20]:
t=0
for i in df_2022['actor_2_name']:
    t=t+1
    print(t,i)

1 Penélope Cruz
2 Danny Trejo
3 Michael Jai White
4 Bruce Willis
5 Mason Gooding
6 Selena Gomez
7 Frank Grillo
8 Mena Massoud
9 Tom Lewis
10 Kaya Scodelario
11 Madalen Mills
12 Billy Zane
13 Maddie Ziegler
14 Taylor Lautner
15 Utkarsh Ambudkar
16 nan
17 Steve-O
18 Patrick Wilson
19 Mel Gibson
20 Nolan Gerard Funk
21 Rita Wilson
22 Jake Horowitz
23 Annette Bening
24 Owen Wilson
25 Emmy Raver-Lampman
26 Jenny Slate
27 Griffin Gluck
28 Pico Alexander
29 Kyle Allen
30 Lewis Tan
31 Mark Wahlberg
32 Jane Adams
33 Elsie Fisher
34 Kelly Reilly
35 Brian Hull
36 Haley Bennett
37 Nate Mendel
38 Cassi Davis
39 Danny Ramirez
40 Leo Suter
41 Bruce Willis
42 Zoë Kravitz
43 Jodie Turner-Smith
44 Sebastian Stan
45 Bruce Willis
46 Penelope Mitchell
47 Walker Scobell
48 Sandra Oh
49 Amy Smart
50 Scott Wolf
51 Zoey Deutch
52 Ana de Armas
53 Zach Braff
54 Jesse Plemons
55 Jenna Ortega
56 Fivel Stewart
57 Jonny Lee Miller
58 Zoe Renee
59 Bella Thorne
60 Channing Tatum
61 Stephanie Hsu
62 Billy Howle
63 Gera

In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].astype("str").map(lambda x: get_actor3(x))

In [23]:
t=0
for i in df_2022['actor_3_name']:
    t=t+1
    print(t,i)

1 Fan Bingbing
2 Antonio Cupo
3 nan
4 Rob Gough
5 Jenna Ortega
6 Kathryn Hahn
7 Lilly Krug
8 nan
9 Logan Marshall-Green
10 Benjamin Walker
11 Katharine McPhee
12 Tony Todd
13 Julie Bowen
14 Rob Schneider
15 Justina Machado
16 nan
17 Chris Pontius
18 John Bradley
19 Morena Baccarin
20 Jeff Fahey
21 Jaime Camil
22 Luciana VanDette
23 Kenneth Branagh
24 Maluma
25 Taylor John Smith
26 Scott Eastwood
27 Sabrina Carpenter
28 Jacques Colimon
29 nan
30 Lawrence Kao
31 Sophia Ali
32 Kevin Nash
33 Mark Burham
34 Alistair Petrie
35 Gabriel Burrafato
36 Kelvin Harrison Jr.
37 Pat Smear
38 David Mann
39 David Rysdahl
40 Stephen Dorff
41 Luke Wilson
42 Paul Dano
43 Justin H. Min
44 Jonica T. Gibbs
45 Gianni Capaldi
46 Elaine Tan
47 Mark Ruffalo
48 Ava Morse
49 Major Dodson
50 Kaylah Zander
51 Johnny Flynn
52 nan
53 Erika Christensen
54 Jason Segel
55 Martin Henderson
56 Dermot Mulroney
57 Common
58 Amber Gray
59 Jake Weary
60 Daniel Radcliffe
61 Ke Huy Quan
62 Denis O'Hare
63 nan
64 Mason Gooding
65

In [24]:
df_2022

Title  \
0                     The 355   
1    The Legend of La Llorona   
2                The Commando   
3              American Siege   
4                      Scream   
..                        ...   
313            Alice, Darling   
314                       NaN   
315                       NaN   
316                       NaN   
317                       NaN   

                                         Cast and crew  \
0    Simon Kinberg (director/screenplay); Theresa R...   
1    Patricia Harris Seeley (director); José Prende...   
2    Asif Akbar (director); Koji Steven Sakai (scre...   
3    Edward John Drake (director/screenplay); Timot...   
4    Matt Bettinelli-Olpin, Tyler Gillett (director...   
..                                                 ...   
313  Mary Nighy (director); Alanna Francis (screenp...   
314                                                NaN   
315                                                NaN   
316                                                NaN   
317                                                NaN   

                              genres        id  \
0                    Action Thriller  522016.0   
1    Family Animation Fantasy Horror  116322.0   
2              Action Crime Thriller  753232.0   
3          Action Adventure Thriller  766907.0   
4            Horror Mystery Thriller  646385.0   
..                               ...       ...   
313           Thriller Drama Romance  850028.0   
314                      Drama Crime  384717.0   
315                      Drama Crime  384717.0   
316                      Drama Crime  384717.0   
317                      Drama Crime  384717.0   

                             director_name       actor_1_name  \
0                           Simon Kinberg    Jessica Chastain   
1                  Patricia Harris Seeley       Autumn Reeser   
2                              Asif Akbar       Mickey Rourke   
3                       Edward John Drake   Timothy V. Murphy   
4    Matt Bettinelli-Olpin, Tyler Gillett     Melissa Barrera   
..                                     ...                ...   
313                            Mary Nighy       Anna Kendrick   
314                                    nan                nan   
315                                    nan                nan   
316                                    nan                nan   
317                                    nan                nan   

          actor_2_name     actor_3_name  
0        Penélope Cruz     Fan Bingbing  
1          Danny Trejo     Antonio Cupo  
2    Michael Jai White              NaN  
3         Bruce Willis        Rob Gough  
4        Mason Gooding     Jenna Ortega  
..                 ...              ...  
313    Kaniehtiio Horn  Charlie Carrick  
314               None              NaN  
315               None              NaN  
316               None              NaN  
317               None              NaN  

[318 rows x 8 columns]

In [25]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [26]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title','id']]

In [27]:
new_df22

director_name       actor_1_name  \
0                           Simon Kinberg    Jessica Chastain   
1                  Patricia Harris Seeley       Autumn Reeser   
2                              Asif Akbar       Mickey Rourke   
3                       Edward John Drake   Timothy V. Murphy   
4    Matt Bettinelli-Olpin, Tyler Gillett     Melissa Barrera   
..                                     ...                ...   
313                            Mary Nighy       Anna Kendrick   
314                                    nan                nan   
315                                    nan                nan   
316                                    nan                nan   
317                                    nan                nan   

          actor_2_name     actor_3_name                           genres  \
0        Penélope Cruz     Fan Bingbing                  Action Thriller   
1          Danny Trejo     Antonio Cupo  Family Animation Fantasy Horror   
2    Michael Jai White              NaN            Action Crime Thriller   
3         Bruce Willis        Rob Gough        Action Adventure Thriller   
4        Mason Gooding     Jenna Ortega          Horror Mystery Thriller   
..                 ...              ...                              ...   
313    Kaniehtiio Horn  Charlie Carrick           Thriller Drama Romance   
314               None              NaN                      Drama Crime   
315               None              NaN                      Drama Crime   
316               None              NaN                      Drama Crime   
317               None              NaN                      Drama Crime   

                  movie_title        id  
0                     The 355  522016.0  
1    The Legend of La Llorona  116322.0  
2                The Commando  753232.0  
3              American Siege  766907.0  
4                      Scream  646385.0  
..                        ...       ...  
313            Alice, Darling  850028.0  
314                       NaN  384717.0  
315                       NaN  384717.0  
316                       NaN  384717.0  
317                       NaN  384717.0  

[318 rows x 7 columns]

In [28]:
new_df22['director_name']=new_df22['director_name'].apply(lambda x:x.replace(" ",""))
new_df22['actor_1_name']=new_df22['actor_1_name'].apply(lambda x:x.replace(" ",""))
new_df22['actor_2_name']=new_df22['actor_2_name'].astype("str").apply(lambda x:x.replace(" ",""))
new_df22['actor_3_name']=new_df22['actor_3_name'].astype("str").apply(lambda x:x.replace(" ",""))

In [29]:
new_df22 = new_df22.dropna(how='any')

In [30]:
new_df22['tag'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']
new_df22['tag'] =new_df22['tag'].str.lower()

In [31]:
new_df22=new_df22[["id","movie_title","tag"]]
new_df22['id']=new_df22['id'].astype("int")
new_df22

id                                   movie_title  \
0    522016                                       The 355   
1    116322                      The Legend of La Llorona   
2    753232                                  The Commando   
3    766907                                American Siege   
4    646385                                        Scream   
..      ...                                           ...   
309  696157  Whitney Houston: I Wanna Dance with Somebody   
310  800815                             The Pale Blue Eye   
311  777245                                 Women Talking   
312  937278                             A Man Called Otto   
313  850028                                Alice, Darling   

                                                   tag  
0    jessicachastain penélopecruz fanbingbing simon...  
1    autumnreeser dannytrejo antoniocupo patriciaha...  
2    mickeyrourke michaeljaiwhite nan asifakbar act...  
3    timothyv.murphy brucewillis robgough edwardjoh...  
4    melissabarrera masongooding jennaortega mattbe...  
..                                                 ...  
309  naomiackie stanleytucci ashtonsanders kasilemm...  
310  christianbale harrymelling gilliananderson sco...  
311  rooneymara clairefoy jessiebuckley sarahpolley...  
312  tomhanks marianatreviño rachelkeller marcforst...  
313  annakendrick kaniehtiiohorn charliecarrick mar...  

[313 rows x 3 columns]

## Extracting features of 2021 movies from Wikipedia

In [32]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [33]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [34]:
df

Opening  Opening.1                 Title  \
0      J A N U A R Y          6                 M3GAN   
1      J A N U A R Y          6           The Old Way   
2      J A N U A R Y         11  The Devil Conspiracy   
3      J A N U A R Y         13                 Plane   
4      J A N U A R Y         13           House Party   
..               ...        ...                   ...   
339  D E C E M B E R         22                Memory   
340  D E C E M B E R         25      The Color Purple   
341  D E C E M B E R         25  The Boys in the Boat   
342  D E C E M B E R         25               Ferrari   
343  D E C E M B E R         29            Good Grief   

                                    Production company  \
0    Universal Pictures / Blumhouse Productions / A...   
1                           Saban Films / Saturn Films   
2                                 Samuel Goldwyn Films   
3    Lionsgate / MadRiver Pictures / Di Bonaventura...   
4              Warner Bros. Pictures / New Line Cinema   
..                                                 ...   
339                       Ketchup Entertainment / Mubi   
340  Warner Bros. Pictures / Amblin Entertainment /...   
341          Metro-Goldwyn-Mayer / Smokehouse Pictures   
342            Neon / STXfilms / Ketchup Entertainment   
343                                            Netflix   

                                         Cast and crew   Ref.  
0    Gerard Johnstone (director); Akela Cooper (scr...    [3]  
1    Brett Donowho (director); Carl W. Lucas (scree...    [4]  
2    Nathan Frankowski (director); Ed Alan (screenp...    [5]  
3    Jean-François Richet (director); Charles Cummi...    [6]  
4    Calmatic (director); Jamal Olori, Stephen Glov...    [7]  
..                                                 ...    ...  
339  Michel Franco (director/screenplay); Jessica C...  [326]  
340  Blitz Bazawule (director), Marcus Gardley (scr...  [327]  
341  George Clooney (director), Mark L. Smith (scre...  [328]  
342  Michael Mann (director), Troy Kennedy Martin (...  [329]  
343  Dan Levy (director/screenplay); Dan Levy, Ruth...  [330]  

[344 rows x 6 columns]

In [35]:
df['genres'] = df.progress_apply(lambda row: safe_get_genre(str(row['Title']), row.name), axis=1)

 40%|███████████████████████████████▊                                                | 137/344 [03:40<05:14,  1.52s/it]

Error processing index: 135
Title: Shooting Stars
Error: attribute name must be string, not 'int'


100%|████████████████████████████████████████████████████████████████████████████████| 344/344 [09:22<00:00,  1.64s/it]


In [36]:
df['id'] = df.progress_apply(lambda row: safe_get_id(str(row['Title']), row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 344/344 [00:00<00:00, 2974.26it/s]

Error processing index: 135
Title: Shooting Stars
Error: attribute name must be string, not 'int'


In [37]:
df_2023 = df[['Title','Cast and crew','genres','id']]
df_2023

Title                                      Cast and crew  \
0                   M3GAN  Gerard Johnstone (director); Akela Cooper (scr...   
1             The Old Way  Brett Donowho (director); Carl W. Lucas (scree...   
2    The Devil Conspiracy  Nathan Frankowski (director); Ed Alan (screenp...   
3                   Plane  Jean-François Richet (director); Charles Cummi...   
4             House Party  Calmatic (director); Jamal Olori, Stephen Glov...   
..                    ...                                                ...   
339                Memory  Michel Franco (director/screenplay); Jessica C...   
340      The Color Purple  Blitz Bazawule (director), Marcus Gardley (scr...   
341  The Boys in the Boat  George Clooney (director), Mark L. Smith (scre...   
342               Ferrari  Michael Mann (director), Troy Kennedy Martin (...   
343            Good Grief  Dan Levy (director/screenplay); Dan Levy, Ruth...   

                                      genres         id  
0                     Science Fiction Horror   536554.0  
1                              Western Drama   872954.0  
2    Horror Fantasy Science Fiction Thriller   296271.0  
3                                     Action   646389.0  
4                                     Comedy   632065.0  
..                                       ...        ...  
339                    Action Thriller Crime   818397.0  
340                                    Drama   558915.0  
341                            Drama History   823452.0  
342                            History Drama   365620.0  
343                             Comedy Drama  1036619.0  

[344 rows x 4 columns]

In [38]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (director/acreenplay);" in x:
        return x.split("(director/acreenplay); ")[0]
    elif "(directors/screenplay);" in x:
        return x.split("(directors/screenplay);")[0]
    else:
        return x.split(" (director/screenplay)")[0]


In [39]:
df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))

In [40]:
t=0
for i in df_2023['director_name']:
    t=t+1
    print(t,i)

1 Gerard Johnstone
2 Brett Donowho
3 Nathan Frankowski
4 Jean-François Richet
5 Calmatic
6 John Hyams
7 Sarah Adina Smith
8 Stephen Herek
9 Roxanne Benjamin
10 Nick Johnson, Will Merrick 
11 Florian Zeller
12 Jesse Eisenberg
13 Edward John Drake
14 Jason Eisener
15 Russell Mulcahy
16 Kenya Barris
17 Jason Moore
18 Michael Jacobs
19 Cecilia Miniucchi
20 Deon Taylor
21 M. Night Shyamalan
22 Kyle Marvin
23 Nicolas Harvard
24 Bess Wohl
25 Brett Smith
26 Chris Soriano
27 Steven Soderbergh
28 Benjamin Caron
29 Aline Brosh McKenna
30 Dave Franco
31 Christopher Smith
32 Robbie Banfitch
33 Peyton Reed
34 Elizabeth Banks
35 Christopher Landon
36 Jamie Payne
37 Jon Erwin, Brent McCorkle
38 Miles Warren
39 Colin West
40 Daniel Antebi
41 Michael B. Jordan
42 Guy Ritchie
43 Michael Maren
44 Kurt Wimmer
45 Jamie Dack
46 Alex Heller
47 Yoko Okumura
48 J. Horton
49 Matt Bettinelli-Olpin, Tyler Gillett
50 Scott Beck, Bryan Woods
51 Bobby Farrelly
52 Jingyi Shao
53 Jeffrey A. Smith
54 David F. Sandberg
5

In [41]:
df_2023['director_name'][244]='Mike P. Nelson, Gigi Saul Guerrero, Scott Derrickson, David Bruckner, Natasha Kermani'

In [42]:
t=0
for i in df_2023['director_name']:
    t=t+1
    print(t,i)

1 Gerard Johnstone
2 Brett Donowho
3 Nathan Frankowski
4 Jean-François Richet
5 Calmatic
6 John Hyams
7 Sarah Adina Smith
8 Stephen Herek
9 Roxanne Benjamin
10 Nick Johnson, Will Merrick 
11 Florian Zeller
12 Jesse Eisenberg
13 Edward John Drake
14 Jason Eisener
15 Russell Mulcahy
16 Kenya Barris
17 Jason Moore
18 Michael Jacobs
19 Cecilia Miniucchi
20 Deon Taylor
21 M. Night Shyamalan
22 Kyle Marvin
23 Nicolas Harvard
24 Bess Wohl
25 Brett Smith
26 Chris Soriano
27 Steven Soderbergh
28 Benjamin Caron
29 Aline Brosh McKenna
30 Dave Franco
31 Christopher Smith
32 Robbie Banfitch
33 Peyton Reed
34 Elizabeth Banks
35 Christopher Landon
36 Jamie Payne
37 Jon Erwin, Brent McCorkle
38 Miles Warren
39 Colin West
40 Daniel Antebi
41 Michael B. Jordan
42 Guy Ritchie
43 Michael Maren
44 Kurt Wimmer
45 Jamie Dack
46 Alex Heller
47 Yoko Okumura
48 J. Horton
49 Matt Bettinelli-Olpin, Tyler Gillett
50 Scott Beck, Bryan Woods
51 Bobby Farrelly
52 Jingyi Shao
53 Jeffrey A. Smith
54 David F. Sandberg
5

In [43]:
def get_actor1(x):
    if " (screenplay)" in x:
         if len(x.split("(screenplay); "))>1:
            return ((x.split("(screenplay); ")[-1]).split(", ")[0]) 
         else:
            return np.NaN
        
    elif " (screenwriter)" in x:
        return ((x.split('(screenwriter); ')[-1]).split(",")[0]) 
    
    elif " (director/screenwriter)" in x:
        return ((x.split('(director/screenwriter); ')[-1]).split(",")[0]) 
    
    elif " (co-director/screenplay)" in x:
        return ((x.split('(co-director/screenplay); ')[-1]).split(",")[0])
    
    elif " (director/screenplay)" in x :
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0]) 
    
    elif " (director)"in x:
        if len(x.split('(director); '))>1:
            return ((x.split('(director); ')[-1]).split(",")[0])
        else:
            return np.NaN

    elif "(directors/screenplay);" in x :
        return ((x.split("(directors/screenplay);")[-1]).split(", ")[0]) 

    elif " (director/acreenplay)"in x:
        return ((x.split('(director/acreenplay); ')[-1]).split(",")[0]) 
    
    else:
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0]) 

In [44]:
df_2023['actor_1_name'] = df_2023['Cast and crew'].astype(str).map(lambda x: get_actor1(x))

In [45]:
t=0
for i in df_2023['actor_1_name']:
    t=t+1
    print(t-1,i)

0 Allison Williams
1 Nicolas Cage
2 Alice Orr-Ewing
3 Gerard Butler
4 Tosin Cole
5 Gideon Adlon
6 Anna Konkle
7 Rob Lowe
8 Alisha Wainwright
9  Storm Reid
10 Hugh Jackman
11 Julianne Moore
12 Bruce Willis
13 Dominic Mariche
14 Tyler Posey
15 Jonah Hill
16 Jennifer Lopez
17 Diane Keaton
18 Danny Huston
19 Joseph Sikora
20 Dave Bautista
21 Lily Tomlin
22 Ryan Phillippe
23 Noémie Merlant
24 Gerran Howell
25 Chris Soriano
26 Channing Tatum
27 Julianne Moore
28 Reese Witherspoon
29 Alison Brie
30 Jena Malone
31 Robbie Banfitch
32 Paul Rudd
33 Keri Russell
34 David Harbour
35 Idris Elba
36 Joel Courtney
37 Trevante Rhodes
38 Jim Gaffigan
39 Ben Groh
40 Michael B. Jordan
41 Jason Statham
42 Michael Shannon
43 Elena Kampouris
44 Lily McInerny
45 Alex Heller
46 Midori Francis
47 Rachel Amanda Bryant
48 Melissa Barrera
49 Adam Driver
50 Woody Harrelson
51 Bloom Li
52 Max Ehrich
53 Zachary Levi
54 Keira Knightley
55 Noah Jupe
56 Jane Fonda
57 Kiernan Shipka
58 Skeet Ulrich
59  Mike Faist
60 Keanu

In [46]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    
    elif len((x.split("(directors); ")[-1]).split(", ")) < 2:
        return np.NaN
    
    elif " (screenplay)," in x:
        return ((x.split("(screenplay), ")[-1]).split(", ")[1])
    
    elif " (screenplay);" in x:
        return ((x.split("(screenplay); ")[-1]).split(", ")[1])
    
    elif " (directors/screenplay)" in x:
        return ((x.split("(directors/screenplay); ")[-1]).split(", ")[1])
    
    elif " (director/screenplay)" in x: 
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[1])
    else:
        return ((x.split("(director); ")[-1]).split(", ")[1])

In [47]:
df_2023['actor_2_name'] = df_2023['Cast and crew'].astype(str).map(lambda x: get_actor2(x))

In [48]:
t=0
for i in df_2023['actor_2_name']:
    t=t+1
    print(t-1,i)

0 Violet McGraw
1 Ryan Kiera Armstrong
2 Joe Doyle
3 Mike Colter
4 Jacob Latimore
5 Bethlehem Million
6 Jermaine Fowler
7 Johnny Berchtold
8 Zach Gilford
9 Joaquim de Almeida
10 Laura Dern
11 Finn Wolfhard
12 nan
13 Phoebe Rex
14 Crystal Reed
15 Lauren London
16 Josh Duhamel
17 Richard Gere
18 Radha Mitchell
19 King Bach
20 Jonathan Groff
21 Jane Fonda
22 Kate Bosworth
23 Kit Harington
24 RJ Cyler
25 Christine S. Chang
26 Salma Hayek Pinault
27 Sebastian Stan
28 Ashton Kutcher
29 Jay Ellis
30 Danny Huston
31 Angela Basolis
32 Evangeline Lilly
33 O'Shea Jackson Jr.
34 Anthony Mackie
35 Cynthia Erivo
36 Anna Grace Barlow
37 Jalyn Hall
38 Rhea Seehorn
39 Dion Costelloe
40 Tessa Thompson
41 Aubrey Plaza
42 Kate Hudson
43 Kate Moyer
44 Jonathan Tucker
45 J. Smith-Cameron
46 Jolene Purdy
47 Al Gomez
48 Jasmin Savoy Brown
49 Ariana Greenblatt
50 Kaitlin Olson
51 Ben Wang
52 Katelyn Nacon
53 Asher Angel
54 Carrie Coon
55 Benedict Wong
56 Lily Tomlin
57 Dash Mihok
58 Anne Heche
59 Crystal Reed


In [49]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [50]:
df_2023['actor_3_name'] = df_2023['Cast and crew'].astype(str).map(lambda x: get_actor3(x))

In [51]:
t=0
for i in df_2023['actor_3_name']:
    t=t+1
    print(t-1,i)

0 Amie Donald
1 nan
2 Eveline Hall
3 Yoson An
4 Karen Obilom
5 Marc Menchaca
6 Jillian Bell
7 Kimberly Williams-Paisley
8 Amanda Crew
9 Ken Leung
10 Vanessa Kirby
11 Alisha Boe
12 nan
13 Calem MacDonald
14 Tyler Hoechlin
15 David Duchovny
16 Sônia Braga
17 Susan Sarandon
18 Bob Odenkirk
19 Annie Ilonzeh
20 Ben Aldridge
21 Rita Moreno
22 Ving Rhames
23 Meredith Hagner
24 Ewen Bremner
25 Hillary Soriano
26 nan
27 Justice Smith
28 Jesse Williams
29 Kiersey Clemons
30 Janet Suzman
31 Scott Schamell
32 Jonathan Majors
33 Christian Convery
34 Jahi Di'Allo Winston
35 Andy Serkis
36 Jonathan Roumie
37 Shinelle Azoroh
38 Katelyn Nacon
39 Liz Caribel Sierra
40 Jonathan Majors
41 Josh Hartnett
42 Don Johnson
43 Callan Mulvey
44 Gretchen Mol
45 Steve Buscemi
46 Missi Pyle
47 Gregory Blair
48 Jack Champion
49 Chloe Coleman
50 Ernie Hudson
51 Dexter Darden
52 J. Alphonse Nicholson
53 Jack Dylan Grazer
54 Alessandro Nivola
55 Sian Clifford
56 Malcolm McDowell
57 Charlie Plummer
58 Daniel Diemer
59 De

In [52]:
df_2023 = df_2023.rename(columns={'Title':'movie_title'})

In [53]:
df_2023

movie_title                                      Cast and crew  \
0                   M3GAN  Gerard Johnstone (director); Akela Cooper (scr...   
1             The Old Way  Brett Donowho (director); Carl W. Lucas (scree...   
2    The Devil Conspiracy  Nathan Frankowski (director); Ed Alan (screenp...   
3                   Plane  Jean-François Richet (director); Charles Cummi...   
4             House Party  Calmatic (director); Jamal Olori, Stephen Glov...   
..                    ...                                                ...   
339                Memory  Michel Franco (director/screenplay); Jessica C...   
340      The Color Purple  Blitz Bazawule (director), Marcus Gardley (scr...   
341  The Boys in the Boat  George Clooney (director), Mark L. Smith (scre...   
342               Ferrari  Michael Mann (director), Troy Kennedy Martin (...   
343            Good Grief  Dan Levy (director/screenplay); Dan Levy, Ruth...   

                                      genres         id         director_name  \
0                     Science Fiction Horror   536554.0      Gerard Johnstone   
1                              Western Drama   872954.0         Brett Donowho   
2    Horror Fantasy Science Fiction Thriller   296271.0     Nathan Frankowski   
3                                     Action   646389.0  Jean-François Richet   
4                                     Comedy   632065.0              Calmatic   
..                                       ...        ...                   ...   
339                    Action Thriller Crime   818397.0         Michel Franco   
340                                    Drama   558915.0        Blitz Bazawule   
341                            Drama History   823452.0        George Clooney   
342                            History Drama   365620.0          Michael Mann   
343                             Comedy Drama  1036619.0              Dan Levy   

         actor_1_name          actor_2_name      actor_3_name  
0    Allison Williams         Violet McGraw       Amie Donald  
1        Nicolas Cage  Ryan Kiera Armstrong               NaN  
2     Alice Orr-Ewing             Joe Doyle      Eveline Hall  
3       Gerard Butler           Mike Colter          Yoson An  
4          Tosin Cole        Jacob Latimore      Karen Obilom  
..                ...                   ...               ...  
339  Jessica Chastain       Peter Sarsgaard     Merritt Wever  
340  Fantasia Barrino      Taraji P. Henson   Danielle Brooks  
341     Callum Turner         Joel Edgerton    Peter Guinness  
342       Adam Driver         Penelope Cruz  Shailene Woodley  
343          Dan Levy            Ruth Negga      Himesh Patel  

[344 rows x 8 columns]

In [54]:
new_df23 = df_2023.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title','id']]

In [55]:
new_df23['director_name']=new_df23['director_name'].apply(lambda x:x.replace(" ",""))
new_df23['actor_1_name']=new_df23['actor_1_name'].apply(lambda x:x.replace(" ",""))
new_df23['actor_2_name']=new_df23['actor_2_name'].astype("str").apply(lambda x:x.replace(" ",""))
new_df23['actor_3_name']=new_df23['actor_3_name'].astype("str").apply(lambda x:x.replace(" ",""))

In [56]:
new_df23 = new_df23.dropna(how='any')

In [57]:
new_df23['tag'] = new_df23['actor_1_name'] + ' ' + new_df23['actor_2_name'] + ' '+ new_df23['actor_3_name'] + ' '+ new_df23['director_name'] +' ' + new_df23['genres']
new_df23['tag']=new_df23['tag'].str.lower()
new_df23['id']=new_df23['id'].astype("int")

In [58]:
new_df23=new_df23[["id","movie_title","tag"]]
new_df23

id           movie_title  \
0     536554                 M3GAN   
1     872954           The Old Way   
2     296271  The Devil Conspiracy   
3     646389                 Plane   
4     632065           House Party   
..       ...                   ...   
339   818397                Memory   
340   558915      The Color Purple   
341   823452  The Boys in the Boat   
342   365620               Ferrari   
343  1036619            Good Grief   

                                                   tag  
0    allisonwilliams violetmcgraw amiedonald gerard...  
1    nicolascage ryankieraarmstrong nan brettdonowh...  
2    aliceorr-ewing joedoyle evelinehall nathanfran...  
3    gerardbutler mikecolter yosonan jean-françoisr...  
4    tosincole jacoblatimore karenobilom calmatic c...  
..                                                 ...  
339  jessicachastain petersarsgaard merrittwever mi...  
340  fantasiabarrino tarajip.henson daniellebrooks ...  
341  callumturner joeledgerton peterguinness george...  
342  adamdriver penelopecruz shailenewoodley michae...  
343  danlevy ruthnegga himeshpatel danlevy comedy d...  

[343 rows x 3 columns]

In [59]:
my_df = pd.concat([new_df22, new_df23], ignore_index=True)

In [60]:
my_df

id               movie_title  \
0     522016                   The 355   
1     116322  The Legend of La Llorona   
2     753232              The Commando   
3     766907            American Siege   
4     646385                    Scream   
..       ...                       ...   
651   818397                    Memory   
652   558915          The Color Purple   
653   823452      The Boys in the Boat   
654   365620                   Ferrari   
655  1036619                Good Grief   

                                                   tag  
0    jessicachastain penélopecruz fanbingbing simon...  
1    autumnreeser dannytrejo antoniocupo patriciaha...  
2    mickeyrourke michaeljaiwhite nan asifakbar act...  
3    timothyv.murphy brucewillis robgough edwardjoh...  
4    melissabarrera masongooding jennaortega mattbe...  
..                                                 ...  
651  jessicachastain petersarsgaard merrittwever mi...  
652  fantasiabarrino tarajip.henson daniellebrooks ...  
653  callumturner joeledgerton peterguinness george...  
654  adamdriver penelopecruz shailenewoodley michae...  
655  danlevy ruthnegga himeshpatel danlevy comedy d...  

[656 rows x 3 columns]

In [61]:
my_df.isnull().sum()

id             0
movie_title    0
tag            0
dtype: int64

In [62]:
my_df.shape

(656, 3)

In [63]:
old_df = pd.read_csv('till 2021.csv')

In [72]:
old_df

movie_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
6200                            The King's Man   
6201                    The Tragedy of Macbeth   
6202                      A Journal for Jordan   
6203                         American Underdog   
6204                                   Memoria   

                                                    tag        id  
0     cchpounder joeldavidmoore wesstudi jamescamero...   19995.0  
1     johnnydepp orlandobloom jackdavenport goreverb...     285.0  
2     christophwaltz rorykinnear stephaniesigman sam...  206647.0  
3     tomhardy christianbale josephgordon-levitt chr...   49026.0  
4     darylsabara samanthamorton pollywalker andrews...   49529.0  
...                                                 ...       ...  
6200  ralphfiennes gemmaarterton rhysifans matthewva...  476669.0  
6201  denzelwashington francesmcdormand bertiecarvel...  591538.0  
6202  michaelb.jordan chantéadams jalonchristian den...  501841.0  
6203  zacharylevi annapaquin dennisquaid erwinbrothe...  673309.0  
6204  tildaswinton elkindíaz jeannebalibar apichatpo...  511819.0  

[6205 rows x 3 columns]

In [75]:
final_df = pd.concat([old_df, my_df], ignore_index=True)

In [76]:
final_df['id']=final_df['id'].astype("int")
final_df

movie_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
6856                                    Memory   
6857                          The Color Purple   
6858                      The Boys in the Boat   
6859                                   Ferrari   
6860                                Good Grief   

                                                    tag       id  
0     cchpounder joeldavidmoore wesstudi jamescamero...    19995  
1     johnnydepp orlandobloom jackdavenport goreverb...      285  
2     christophwaltz rorykinnear stephaniesigman sam...   206647  
3     tomhardy christianbale josephgordon-levitt chr...    49026  
4     darylsabara samanthamorton pollywalker andrews...    49529  
...                                                 ...      ...  
6856  jessicachastain petersarsgaard merrittwever mi...   818397  
6857  fantasiabarrino tarajip.henson daniellebrooks ...   558915  
6858  callumturner joeledgerton peterguinness george...   823452  
6859  adamdriver penelopecruz shailenewoodley michae...   365620  
6860  danlevy ruthnegga himeshpatel danlevy comedy d...  1036619  

[6861 rows x 3 columns]

In [77]:
final_df.isnull().sum()

movie_title    0
tag            0
id             0
dtype: int64

In [78]:
final_df.to_csv('till 2023.csv',index=False)